### Практическое задание к уроку 6
1. Реализовать адаптивный бустинг использующий Логистическую Регрессию и меру ошибок LogLoss. Сравнить с точностью адаптивного бустинга на деревьях решений. Для сбора предсказаний можно использовать ту же функцию predict что и для бустинга на деревьях
2. Реализовать специальную функцию predict для бустинга на логистической регрессии выводящую предсказания по формуле:$ Predictions=sign(Score_{bust})$, где sign равен единице для положительных и нулю для отрицательных значений, а $ Score_{bust}= \sum \alpha_iScore_i$. Баллы выдаваемые каждой моделью $Score_i$ можно найти при помощи вызова метода decision_function на моделе

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt

from sklearn import model_selection
import numpy as np

import warnings

In [2]:
def get_error(pred, y):
    return np.sum((pred != y).astype(int)) / len(y)

In [3]:
# LogLoss
def log_loss(pred, y): 
    return -np.sum(y*np.log2(pred)+(1-y)*np.log2(1-pred))/len(y)

In [4]:
# Адаптивный бустинг на деревьях решений
def adaboost(X, y, N):

    # Размер выборки
    n_objects = len(X)

    # Запишем количество классов в переменную
    n_classes = len(np.unique((y)))

    # Начальные веса деревьев
    w = np.ones(n_objects) / n_objects

    # Деревья с весами будем записывать в список
    models = []

    for n in range(N):
        # Зададим дерево и обучим его
        clf = DecisionTreeClassifier(max_depth=1)
        clf.fit(X, y, w)

        predictions = clf.predict(X)
        e = get_error(predictions, y)
        # отбросим дерево, если его ошибка больше 0.5
        # Запишем условие в общем виде (применимо к небинарным классификаторам)
        if e >= 1 - 1/n_classes: 
            break

        # Вычислим вес для дерева
        alpha = 0.5 * np.log((1 - e) / e)

        # Найдем индексы правильно классифицированных элементов
        match = predictions == y

        # Увеличим веса для неправильно классифицированных элементов
        w[np.logical_not(match)] *= np.exp(alpha)
        w[match] *= np.exp(-alpha)

        # Нормализуем веса
        w /= w.sum()

        # Добавим дерево с весом в список
        models.append((alpha, clf))
    
    return models

In [6]:
def predict(X, models, n_classes = 2):
    
    n_objects = len(X)
    
    # вначале обозначим предсказание нулевым массивом
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, clf in models:
        prediction = clf.predict(X)
        # Для каждого предсказания будем прибавлять alpha к
        # элементу с индексом предсказанного класса
        y_pred[range(n_objects), prediction] += alpha
    
    # выберем индексы с максимальными суммарными весами -
    # получим предсказанные алгоритмом классы
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

In [5]:
# Адаптивный бустинг на логистической регрессии
def adaboost_log(X, y, N):

    # Размер выборки
    n_objects = len(X)

    # Запишем количество классов в переменную
    n_classes = len(np.unique((y)))

    # Начальные веса деревьев
    w = np.ones(n_objects) / n_objects

    # Деревья с весами будем записывать в список
    models = []

    for n in range(N):
        # Зададим дерево и обучим его
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            clf = LogisticRegression()
            clf.fit(X, y, w)

        predictions = clf.predict(X)
        probabilities = clf.predict_proba(X)[:, 1]
        e = log_loss(probabilities, y)
        # отбрасываю модель, если её ошибка больше 0.5
        # Сравнение разных порогов отсечения моделей показало, что порог 0.5 является оптимальным
        # При его уменьшении сильно падает качество предсказания, а увеличение не влияет на результат
        if e >= 0.5: 
            break

        # Вычислим вес для дерева
        alpha = 0.5 * np.log((1 - e) / e)

        # Найдем индексы правильно классифицированных элементов
        match = predictions == y

        # Увеличим веса для неправильно классифицированных элементов
        w[np.logical_not(match)] *= np.exp(alpha)
        w[match] *= np.exp(-alpha)

        # Нормализуем веса
        w /= w.sum()

        # Добавим дерево с весом в список
        models.append((alpha, clf))
    
    return models

In [19]:
# функция predict, использующая метод decision_function модели LogisticRegression
def predict_with_decision_func(X, models, n_classes = 2):
    
    n_objects = len(X)
    
    # вначале обозначим предсказание нулевым массивом
    y_pred = np.zeros((n_objects, n_classes))
    score_bust = np.zeros(n_objects)
    
    for alpha, clf in models:
        score = clf.decision_function(X)
        score_bust += (alpha * score)
    
    y_pred = np.where(score_bust >= 0, 1, 0)
    
    return y_pred

Загружаю датасет и обучаю 2 модели адаптичного бустинга: на деревьях решений и на логистической регрессии.

In [16]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=0)

In [17]:
N = 50

models_tree = adaboost(X_train, y_train, N)
models_log = adaboost_log(X_train, y_train, N)

In [20]:
print(f'Точность бустинга на деревьях решений на обучающей выборке: {(1 - get_error(predict(X_train, models_tree), y_train)) * 100:.3f}')
print(f'Точность бустинга на логистической регрессии на обучающей выборке: {(1 - get_error(predict_with_decision_func(X_train, models_log), y_train)) * 100:.3f}')

Точность бустинга на деревьях решений на обучающей выборке: 97.183
Точность бустинга на логистической регрессии на обучающей выборке: 93.662


In [21]:
print(f'Точность бустинга на деревьях решений на тестовой выборке: {(1 - get_error(predict(X_test, models_tree), y_test)) * 100:.3f}')
print(f'Точность бустинга на логистической регрессии на тестовой выборке: {(1 - get_error(predict_with_decision_func(X_test, models_log), y_test)) * 100:.3f}')

Точность бустинга на деревьях решений на тестовой выборке: 95.105
Точность бустинга на логистической регрессии на тестовой выборке: 92.308


Из полученных оценок точности хорошо видно, что адаптивный бустинг на деревьях решений переобучается на обучающей выборке, в то время как адаптивный бустинг на логистической регрессии показывает практически идентичный результат как на обучающей выборке, так и на тестовой. Не смотря на переобучение, бустинг на деревьях решений показал результат лучше, чем бустинг на логистической регресии.